In [55]:
import re
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [1]:
input_file_path = 'data/the-verdict.txt'

In [3]:
with open(input_file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

print(f"Length of raw text: {len(raw_text)} characters")
print(raw_text[:100])  

Length of raw text: 20479 characters
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


#### Raw text to tokens

In [15]:
preprocessed = re.split(r'([,.;?_!"()\']|--|\s)', raw_text) # split on commas, periods, question amrks, quotation marks, double dashes and whitespace
preprocessed = [item for item in preprocessed if item.strip()] # remove whitespace-only tokens
len(preprocessed)

4670

In [16]:
preprocessed[:30]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

#### Token to Token IDs

In [17]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(f"Vocab size: {vocab_size} tokens")

Vocab size: 1146 tokens


In [18]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    if i < 50:
        print(item)
    else:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindle:', 43)
('Grindles', 44)
('HAD', 45)
('Had', 46)
('Hang', 47)
('Has', 48)
('He', 49)


#### Text tokenizer class

In [19]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer: token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        token_ids = [self.str_to_int[token] for token in preprocessed]
        return token_ids
    
    def decode(self, token_ids):
        text = ' '.join([self.int_to_str[token_id] for token_id in token_ids])
        text = re.sub(r'\s+([,.;?_!"()\'])', r'\1', text) # remove space before punctuation
        return text


In [20]:
tokenizer = SimpleTokenizerV1(vocab)

In [21]:
text = """"It's the last he painted, you know," Mrs. Gisburn said"""
ids = tokenizer.encode(text)
print(ids)

[1, 57, 2, 861, 1002, 607, 536, 755, 5, 1142, 601, 5, 1, 68, 7, 38, 862]


In [22]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said


#### Tokenizer handles unknown words

In [24]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

In [25]:
print(len(vocab.items()))

1148


In [26]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1143)
('your', 1144)
('yourself', 1145)
('<|endoftext|>', 1146)
('<|unk|>', 1147)


In [27]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer: token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        token_ids = [self.str_to_int[token] for token in preprocessed]
        return token_ids
    
    def decode(self, token_ids):
        text = ' '.join([self.int_to_str[token_id] for token_id in token_ids])
        text = re.sub(r'\s+([,.;?_!"()\'])', r'\1', text) # remove space before punctuation
        return text

In [36]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join([text1, text2])
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [37]:
tokenizer = SimpleTokenizerV2(vocab)

In [38]:
print(tokenizer.encode(text))

[1147, 5, 357, 1142, 634, 989, 10, 1146, 56, 1002, 970, 998, 730, 1002, 1147, 7]


In [39]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


#### Byte Pair encoding

In [41]:
tokenizer = tiktoken.get_encoding("gpt2")

In [42]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562, 13]


In [43]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


#### Data sampling with sliding window

In [44]:
with open(input_file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

enc_text = tokenizer.encode(raw_text)
print(f"Length of tokenized text: {len(enc_text)} tokens")

Length of tokenized text: 5145 tokens


In [45]:
enc_sample = enc_text[50:]

In [47]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]
print("Input sequence (x):", x)
print("Target sequence (y):     ", y)

Input sequence (x): [290, 4920, 2241, 287]
Target sequence (y):      [4920, 2241, 287, 257]


In [48]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "->", desired)

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [53]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    

    print(tokenizer.decode(context), "->", tokenizer.decode([desired]))

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


#### Dataset for batched inputs and targets

In [56]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [57]:
def create_dataloader_v1(text, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [ ]:
with open(input_file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False, drop_last=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [59]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [61]:
with open(input_file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False, drop_last=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


#### Token embeddings


In [65]:
input_ids = torch.tensor([2,3,5,1])
vocab_size = 6
output_dim = 3


In [63]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [64]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [66]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


#### Positional encoding

In [67]:
output_dim = 256
vocab_size = 50257  # GPT-2 vocabulary size
token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=output_dim)

In [69]:
max_length = 4
batch_size = 8
dataloader = create_dataloader_v1(raw_text, batch_size=batch_size, max_length=max_length, stride=max_length, shuffle=False, drop_last=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs: \n", inputs)
print("Inputs shape:", inputs.shape)  # Expected: (batch_size, max_length)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape: torch.Size([8, 4])


In [70]:
token_embeddings = token_embedding_layer(inputs)
print("Token Embeddings shape:", token_embeddings.shape)  

Token Embeddings shape: torch.Size([8, 4, 256])


In [72]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(num_embeddings=context_length, embedding_dim=output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print("Positional Embeddings shape:", pos_embeddings.shape)

Positional Embeddings shape: torch.Size([4, 256])


In [75]:
input_embeddings = token_embeddings + pos_embeddings
print("Input Embeddings shape:", input_embeddings.shape)

Input Embeddings shape: torch.Size([8, 4, 256])
